# Decorators


### What is a decorator?

"Decorator" is the name of a pattern, where a "decorator" allows you to add behaviour to a class, method or function without modifying the original code of these, or their interface. 
In python decorating is usually achieved by wrapping the decorator functionality around the "decoratee" function. This is possible because functions can be passed along as parameters to other functions.

#### Example of function being passed as parameter:

In [1]:
def doSomething():
    print("I did my thing!")

def iDoSomethingBeforeAndAfter(func):
    print("This happened before the function i got passed.")
    func()
    print("This happened after the function i got passed.")

iDoSomethingBeforeAndAfter(doSomething)

This happened before the function i got passed.
I did my thing!
This happened after the function i got passed.


### Decorator syntax:

This however doesn't cut it as a decorator, becuase the interface is different from the function it addds functionality to. In order to comply with the original we have to use a function wrapper. This also allows for some pretty slick syntax!

In [13]:
def my_decorator(func):
    def wrapper():
        print("This happened before the function i got passed.")
        func()
        print("This happened after the function i got passed.")
    return wrapper
@my_decorator
def doSomething():
    print("I did my thing!")

doSomething()

This happened before the function i got passed.
I did my thing!
This happened after the function i got passed.


### Handling arguments:

That looks pretty good! By returning an inner function, instead of just executing code, and using the @ notation on the line above the definition of the decoratee, we're able to add functionality to the decoratee without changing it's interface! That way the wrapper function is called with the decoratee everytime the decoratee is called.
However this decorator is pretty brittle, as it wouldn't work with any functions having arguments passed to them. Lets fix it:

In [12]:
def my_decorator(func):
    def wrapper():
        print("This happened before the function i got passed.")
        func()
        print("This happened after the function i got passed.")
    return wrapper
@my_decorator
def doSomething(message):
    print(message)

doSomething("Hi")

TypeError: wrapper() takes 0 positional arguments but 1 was given

This can be fixed adding \*args and \*\*kwargs as paramaters of the wrapper and the func() call:

In [1]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("This happened before the function i got passed.")
        func(*args,**kwargs)
        print("This happened after the function i got passed.")
    return wrapper
@my_decorator
def doSomething(message):
    print(message)

doSomething("Hi")

This happened before the function i got passed.
Hi
This happened after the function i got passed.


### Returning values:

This still not perfect though! If the decoratee returns a value we currently won't be receiving it!

In [18]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("This happened before the function i got passed.")
        func(*args,**kwargs)
        print("This happened after the function i got passed.")
    return wrapper
@my_decorator
def doSomething(message):
    print(message)
    return len(message)

type(doSomething("Hi"))

This happened before the function i got passed.
hi
This happened after the function i got passed.


NoneType

That sholuld've been an int! However the wrapper never returns the length of the message. Let's change that!

In [27]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("This happened before the function i got passed.")
        func(*args,**kwargs)
        print("This happened after the function i got passed.")
        return func(*args,**kwargs)
    return wrapper
@my_decorator
def doSomething(message):
    print(message)
    return len(message)

type(doSomething("Hi"))

This happened before the function i got passed.
hi
This happened after the function i got passed.
hi


int

Keep in mind that the return statement will be the last thing excuted, so all your decorator behaviour should be before the return statement. In this example it causes the message to be printed twice, becuase i want to keep the second print from the decorator, so the decoratee gets called twice, first when it's called between the 2 prints, and another time when it returns it's value.

### Introspection:
Our current decorator does confuse our doSomething function quite a bit though! It currently thinks that it is a wrapper.

In [30]:
help(doSomething)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In order to fix this introspection issue we will have to use the modul functools.

In [32]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("This happened before the function i got passed.")
        func(*args,**kwargs)
        print("This happened after the function i got passed.")
        return func(*args,**kwargs)
    return wrapper
@my_decorator
def doSomething(message):
    print(message)
    return len(message)

help(doSomething)

Help on function doSomething in module __main__:

doSomething(message)



Using the @functools.wraps(func) decorator from the functools module we make sure that the wrappers meta data is the same as the function it is decorating, and thereby ensuring that doSomething doesn't think it's a wrapper!

## Plug-in structure
Decorators don't need to add functionality to the functions they're decorating. They can also just add something to a list, dict or namespace. These kind of decorators can enable a light weight plug-in structure.
### Plug-in example:


In [33]:
PLUGIN_OPERATION=dict()

def register(func):
    PLUGIN_OPERATION[func.__name__]=func
    return func

@register
def add(a,b):
    return a+b
@register
def sub(a,b):
    return a-b
@register
def mult(a,b):
    return a*b

PLUGIN_OPERATION

{'add': <function __main__.add(a, b)>,
 'sub': <function __main__.sub(a, b)>,
 'mult': <function __main__.mult(a, b)>}

By decorating the add, sub and mult function with register, they automaticly gets added to the plug-in dictionary! Now we can access the operation from the dict, using the function names as keys.

In [36]:
add_func=PLUGIN_OPERATION.get("add","")
add_func(2,3)

5